# Digital Twin Fairways
This notebook provides an example on how to use the digital twin backend. It runs a simulation based on fairway information system of the Netherlands. You can define sites (with cargo), climate conditions and ships. Ships will transport the goods from A to B.

In [1]:
import sys
sys.path.append(r"D:\01. Projecten\[130878] DTV vaarwegen\digitaltwin-waterway\dtv_backend")

import datetime

import geojson
import simpy
import time
import json
import shapely
import pandas as pd
import networkx as nx
from networkx.readwrite import json_graph

# library to load the fairway information network
import dtv_backend.fis
# the simpy processes and objects
import dtv_backend.simple
import dtv_backend.network.network_utilities

# reload for debugging purposes
%load_ext autoreload
%autoreload 2

## Input
You can define your input in a json configuration file. The relevant parts are sites, fleet and climate.

In [2]:
# example input
# with open('../dtv_backend/tests/test-01/config.json') as f:
#     config = geojson.load(f)
with open('../../dtv_backend/tests/test-berth/config.json') as f:
    config = geojson.load(f)
# with open('../dtv_backend/tests/test-03/config.json') as f:
#     config = geojson.load(f)
# with open('../dtv_backend/tests/test-04/config.json') as f:
#     config = geojson.load(f)
[ship['properties']['name'] for ship in config['fleet']]
# config['climate'] = {"lobith": 1500}

['Vessel A', 'Vessel B', 'Vessel C', 'Vessel D', 'Vessel E', 'Vessel F']

## Simulation environment
Setup an simulation environment with a time that starts today. You can also choose a date in the past or future. Using the date 0 won't work on windows. 

In [4]:
# Initialize an environment with a real time
now = datetime.datetime.now()
initial_time = now.timestamp()
env = simpy.Environment(initial_time=initial_time)
env.epoch = now

## Network
Here we load the digital twin network. The topological fairway network derived from the Dutch Fairway Information System. The data is processed to be topological connected and usable for transport network analysis.  

In [5]:
## TEMPORARILY LOAD LOCAL FILE ##
def read_json_file(filename):
    with open(filename) as f:
        js_graph = json.load(f)
    return json_graph.node_link_graph(js_graph)

file = r"D:\01. Projecten\[130878] DTV vaarwegen\02. Data\network_digital_twin_v0.3.json"
G = read_json_file(file)

for n in G.nodes:
    G.nodes[n]['geometry'] = shapely.geometry.Point(G.nodes[n]['X'], G.nodes[n]['Y'])
for e in G.edges:
    edge = G.edges[e]
    edge['geometry'] = shapely.wkt.loads(edge['Wkt'])

env.FG = G

## Connect ports to simulation environment
We have a few different entities defined. The port contains cargo and a crane. The crane can be used to load and unload cargo from ships. The site objects (later to be extended with sluices, stopping areas, etc) are geojson features. 

In [6]:
ports = []
for site in config['sites']:
    port = dtv_backend.simple.Port(env, **site['properties'], **site)
    ports.append(port)

## Connect ships to simulation environment
The ships can also contain cargo. The ships can move over the graph. They are instances of the prototype ships from the Rijkswaterstaat ship dataset. You can have multiple copies of the same ship. All ships can work at the same time. 

In [7]:
ships = []
for ship in config['fleet']:
    kwargs = {}
    kwargs.update(ship)
    kwargs.update(ship['properties'])
    # the ship needs to know about the climate
    if 'climate' in config:
        kwargs['climate'] = config['climate']
    ship = dtv_backend.simple.Ship(env, **kwargs)
    ships.append(ship)

## Route from src to dst

In [9]:
# determine route
src = ports[0].node
dst = ports[1].node

path = dtv_backend.fis.shorted_path(env.FG, src, dst)

## Find berths within given distance from route

In [61]:
# note: could be converted into a function, e.g. find_berths(graph, path, max_dist)
max_dist = 1000 # in meters?
graph = env.FG

# loop the nodes in the route
# check for nodes within thie specified distance
# select berths only
berths = set()
for node in path:
    new_graph = nx.generators.ego_graph(graph,
                                        node,
                                        radius=max_dist,
                                        distance='length_m')
    candidate_berths = [n for n in new_graph.nodes if n.startswith('Berth')]
    berths = berths.union(set(candidate_berths))

## Remove Berths which require additional lock passings

In [87]:
#TODO

## Remove Berths for which dimensions do not comply

In [88]:
#TODO

## Determine ETA per node

In [90]:
def compute_path_length(path):
    """ aux fcn to compute distance of a path """
    total_distance = 0
    for e in zip(path[:-1], path[1:]):
        distance = env.FG.edges[e]['length_m']
        total_distance += distance
    return total_distance

In [110]:
graph = env.FG
src_node = path[0]
dst_node = path[-1]
berth_nodes = berths
time_now = env.epoch # to be updated with env.now
mean_speed = 3 # m/s?

# define the candidate nodes as the union of berths and final destination
candidate_nodes = berths.union([dst_node])

# compute the ETA of all candidate nodes from the current node, for the given speed
rows = []
for node in candidate_nodes:
    # get the path from src_node to node and compute distance and speed-based duration
    path_src_to_node = dtv_backend.fis.shorted_path(graph, src_node, node)
    distance_src_to_node = compute_path_length(path_src_to_node)
    duration_src_to_node = distance_src_to_node / mean_speed
    
    # compute the distance from node to destination
    path_node_to_dst = dtv_backend.fis.shorted_path(graph, node, dst_node)
    distance_node_to_dst = compute_path_length(path_node_to_dst)
    
    # define a row to add to rows
    row = {'berth': node,
           'distance from src': distance_src_to_node,
           'duration from src': duration_src_to_node,
           'eta': time_now + datetime.timedelta(seconds=duration_src_to_node),
           'distance to dst': distance_node_to_dst}
    
    # add the row
    rows.append(row)

# convert to dataframe
df = pd.DataFrame(rows)

# compute the distance of the direct path from src to dst
direct_path = dtv_backend.fis.shorted_path(graph, src_node, dst_node)
direct_distance = compute_path_length(direct_path)

# remove berths which take us away from the destination
df = df.loc[df['distance to dst']<=direct_distance]

## Select the berth which is farthest within a time limit

In [141]:
max_timestamp = datetime.datetime(2022, 5, 5, 22, 0) #dummy

# check if berths are within the time range
df['feasible'] = df['eta'] <= max_timestamp

# get the berth which minimizes the distance to the destination
if sum(df['feasible']) == 0:
    # select the one in minimal time?
    best_berth = df.loc[df['eta'].idxmin(), 'berth']
else:
    df_feas = df.loc[df['feasible']]
    best_berth = df_feas.loc[df_feas['distance to dst'].idxmin(), 'berth']

# SNIPPETS

## ETA per node

In [30]:
ship = ships[0]
total_distance = 0
total_duration = 0
rows = []
for e in zip(path[:-1], path[1:]):
    distance = env.FG.edges[e]['length_m']
    total_distance += distance
    total_duration = total_distance / ship.speed
    
    row = {'e': e,
           'n': e[1],
           'duration': distance / ship.speed,
           'distance': distance,
           'total_duration': total_duration,
           'total_distance': total_distance}
    
    rows.append(row)

In [42]:
df = pd.DataFrame(rows)
df['total_duration_hrs'] = df['total_duration'] / 3600

In [44]:
max_sail_seconds = 3600*8
df.query(f'total_duration <= {max_sail_seconds}')

,e,n,duration,distance,total_duration,total_distance,total_duration_hrs
0,"(22161408, 22161426)",22161426,112.527758,337.583275,112.527758,337.583275,0.031258
1,"(22161426, B45863_B)",B45863_B,772.833097,2318.499292,885.360856,2656.082567,0.245934
2,"(B45863_B, B45863_A)",B45863_A,3.686239,11.058718,889.047095,2667.141285,0.246958
3,"(B45863_A, 8866170)",8866170,1485.041916,4455.125747,2374.089011,7122.267032,0.659469
4,"(8866170, B15339_B)",B15339_B,2668.576263,8005.728788,5042.665274,15127.995821,1.400740
5,"(B15339_B, B15339_A)",B15339_A,3.689199,11.067598,5046.354473,15139.063419,1.401765
6,"(B15339_A, 8864161)",8864161,575.674888,1727.024664,5622.029361,16866.088083,1.561675
7,"(8864161, 8861414)",8861414,389.135638,1167.406913,6011.164998,18033.494995,1.669768
8,"(8861414, 8867154)",8867154,129.215393,387.646180,6140.380392,18421.141176,1.705661
9,"(8867154, 8867029)",8867029,2581.453398,7744.360193,8721.833789,26165.501368,2.422732


## Times to all berths

Snippet to compute an overview of the distance of any current location to all Berths, and from the berth to destination

In [57]:
def compute_path_length(path):
    """ aux fcn to compute distance of a path """
    total_distance = 0
    for e in zip(path[:-1], path[1:]):
        distance = env.FG.edges[e]['length_m']
        total_distance += distance
    return total_distance

def evaluate_berths(berths, current_node, dst_node):
    """ fcn to compute the distances from a current node to each
    berth in a list, and from these berths to the destination """
    rows = []
    for berth in berths:
        path_to_berth = dtv_backend.fis.shorted_path(env.FG, current_node, berth)
        path_to_dst = dtv_backend.fis.shorted_path(env.FG, berth, dst)

        row = {'berth': berth,
               'dist_to_berth': compute_path_length(path_to_berth),
               'dist_to_dst': compute_path_length(path_to_dst)}

        rows.append(row)
    
    return pd.DataFrame(rows)

def determine_berths_to_visit(berths, src_node, dst_node, max_distance):
    """ determines a route with berths from source to destination """
    route = [src_node]
    current_node = src_node
    
    pass
    

In [67]:
# get all berths and distances from and to
current_node = ports[0].node
dst_node = ports[1].node
berths = [n for n in env.FG.nodes if n.startswith('Berth')]

df_distances = evaluate_berths(berths+[dst_node], current_node, dst_node)

In [75]:
max_distance = 3*3600*8
df_feasible = df_distances.query(f'dist_to_berth <= {max_distance}')


# alles wat in een buffer van 1 km van de route ligt
# knikker de paden naar berth eruit waar nog een sluis oid er weer uit
# denk ook eens van achteruit de denken, dst naar src

## Simulation loop
Here we define the major simulation loop. It will run for a number of steps. It will wait for ships to become available. If a ship is available it will get a new assignment.  The assignment consists of a four step procedure. sail to source, load cargo, move to destination, unload cargo. All the activities that are logged in the corresponding object.

We have three stop conditions:
- Time span (2 weeks)
- Number of iterations (n=100)
- All cargo transported 

In [ ]:
# Setup and start the simulation
operator = dtv_backend.simple.Operator(env, ships=ships, **config['operator'])
# The ships do work for the operator
for ship in ships:
    env.process(ship.work_for(operator))
# The opertor plans the work move everything from A to B
env.process(operator.plan(ports[0], ports[1]))
# Run for n days
days = 30
two_weeks = now + datetime.timedelta(days=60)
env.run(until=two_weeks.timestamp())

## Postprocessing
After the simulation is complete we can load the activity logs into a data frame and export them to our favorite formats or make charts. 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

log_df = pd.DataFrame(operator.logbook)
log_df['actor_name'] = log_df['Meta'].apply(lambda x:x['actor'].name)
log_df['state'] = log_df['Meta'].apply(lambda x:x['state'])
gantt_df = pd.DataFrame(
    log_df.pivot(index='ActivityID', columns='state')[
        [('Timestamp', 'START'), ('Timestamp', 'STOP'), ('Message', 'START'), ('actor_name', 'START')]
    ].values,
    columns=['Start', 'Stop', 'Name', 'Actor']
)
# TODO: check why operator cycle ends with NaN
gantt_df = gantt_df.dropna()
# add proper gantt chart headers
gantt_df = gantt_df.query('Name != "Cycle"')
gantt_df = gantt_df.sort_values(['Start', 'Name'])

In [ ]:
# sail_df = log_df[log_df.Message.str.startswith('Sailing')]
# path = sail_df.loc[74]['Meta']['path']
# import dtv_backend.postprocessing
# path_gdf = dtv_backend.postprocessing.path2gdf(path, env.FG)
# path_gdf = dtv_backend.network.network_utilities.sort_path(path_gdf)
# import shapely.ops
# shapely.ops.linemerge(list(path_gdf['geometry']))


In [ ]:
import plotly.express as px
import pandas as pd


fig = px.timeline(gantt_df, x_start="Start", x_end="Stop", y="Name", color="Actor", opacity=0.3)
fig.update_yaxes(autorange="reversed")
fig.show()

In [ ]:
maasvlakte_df = log_df.query('actor_name == "Transferium Maasvlakte"')
basel_df = log_df.query('actor_name == "Basel"').sort_values('Timestamp')
basel_df.iloc[0]['Meta']

In [ ]:
fig, ax = plt.subplots(figsize=(13, 4))
ax.plot(maasvlakte_df.Timestamp, maasvlakte_df.Value, label='Maasvlakte')
ax.plot(basel_df.Timestamp, basel_df.Meta.apply(lambda x: x['destination_level']), label='Basel')
ax.set_xlabel('time')
ax.set_ylabel('cargo [ton]')
ax.legend()

In [ ]:
log_df.iloc[-1].Timestamp - log_df.iloc[0].Timestamp

In [ ]:
df = pd.DataFrame([
    [1000, pd.Timedelta('46 days 09:31:47.926482')],
    [1050, pd.Timedelta('37 days 18:56:43.826629')],
    [1100, pd.Timedelta('29 days 14:26:21.724967')],
    [1150, pd.Timedelta('29 days 11:43:11.457584')],
    [1200, pd.Timedelta('23 days 17:43:01.414319')],
    [1250, pd.Timedelta('21 days 08:33:17.322557')],
    [1300, pd.Timedelta('21 days 06:58:02.005501')],
    [1350, pd.Timedelta('21 days 05:24:09.227142')],
    [1400, pd.Timedelta('14 days 19:41:07.269891')],
    [1500, pd.Timedelta('14 days 18:48:13.765817')],
    [1600, pd.Timedelta('14 days 15:00:29.332012')]
    
], columns=['discharge', 'duration'])
df['days'] = df.duration.astype('timedelta64[s]') / (3600 * 24)

In [ ]:
fig, ax = plt.subplots()
ax.plot(df.discharge, df.days, 'ko-')
ax.set_xlabel('discharge @ Lobith')
ax.set_ylabel('days')
ax.set_title('Scenario Maasvlakte - Basel\n30k tonne, 4 Large Rhine + 2 barge pushed convoy long')


In [ ]:
log_df.head(n=20)[['Message', 'actor_name', 'Timestamp', 'state', 'Meta']]